In [52]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [53]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [54]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Regularización_L2")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/tarea-03/7ee556e9e3f44e9bb30be0f2d7768654



In [55]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [56]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [57]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [58]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 20,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [59]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu', kernel_regularizer = regularizers.L2(0.01))) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_13 (Dense)            (None, 784)               615440    
                                                                 
 dropout_5 (Dropout)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 30)                23550     
                                                                 
 dense_15 (Dense)            (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [61]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/20
5999/6000 [============================>.] - ETA: 0s - loss: 0.5285 - accuracy: 0.8859
Epoch 1: val_loss improved from inf to 0.29192, saving model to best_model.hdf5
6000/6000 [==============================] - 66s 11ms/step - loss: 0.5285 - accuracy: 0.8860 - val_loss: 0.2919 - val_accuracy: 0.9433
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5997/6000 [============================>.] - ETA: 0s - loss: 0.3030 - accuracy: 0.9379
Epoch 2: val_loss improved from 0.29192 to 0.24397, saving model to best_model.hdf5
6000/6000 [==============================] - 59s 10ms/step - loss: 0.3030 - accuracy: 0.9379 - val_loss: 0.2440 - val_accuracy: 0.9529
Epoch 3/20
5997/6000 [============================>.] - ETA: 0s - loss: 0.2533 - accuracy: 0.9499
Epoch 3: val_loss improved from 0.24397 to 0.20831, saving model to best_model.hdf5
6000/6000 [==============================] - 60s 10ms/step - loss: 0.2532 - accuracy: 0.9499 - val_loss: 0.2083 - val_accuracy: 0.9624
Epoch 4/20
6000/6000 [==============================] - ETA: 0s - loss: 0.2243 - accuracy: 0.9581
Epoch 4: val_loss improved from 0.20831 to 0.19148, saving model to best_model.hdf5
6000/6000 [==============================] - 61s 10ms/step - loss: 0.2243 - accuracy: 0.9581 - val_loss: 0.1915 - val_accuracy: 0.9662
Epoch 5/20
5995/6000 [============================>.] - ETA:

In [62]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/7ee556e9e3f44e9bb30be0f2d7768654
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                  : (0.8859666585922241, 0.9793166518211365)
COMET INFO:     batch_accuracy [12000]         : (0.0, 1.0)
COMET INFO:     batch_loss [12000]             : (0.07032331824302673, 3.1708664894104004)
COMET INFO:     epoch_duration [20]            : (57.92180841899972, 69.88847463000002)
COMET INFO:     loss [20]                      : (0.13031037151813507, 0.5284606218338013)
COMET INFO:     val_accuracy [20]              : (0.9433000087738037, 0.9785000085830688)
COMET INFO:     val_loss [20